In [29]:
import numpy as np
from numpy.linalg import inv

predict step: 
returns new state vector X and the new P variance

In [30]:
def predict(F, X, B, u, P, Q):
    # Project the state ahead
    X_new = F*X + B*u
    # Project the error covariance ahead
    P_new = F*P*F.T + Q
    return X_new, P_new

gets a new measurement from the system and refines the guess

In [31]:
def measure(P_new, H, R, X_new, z):
    # Compute the Kalman gain
    K = P_new * H.T * inv(H * P_new * H.T + R)
    # Update estimate with measurement z
    X_new = X_new + K * (z - H * X_new)
    # Update the error covariance
    I = np.eye(P_new.shape[0], P_new.shape[1])
    P_new = (I - K * H) * P_new
    return X_new, P_new, K


# Q3:

In [32]:
class KalmanFilter:
    def __init__(self, F, B, u, Q, H, R, X, P):
        self.F = F  # state transistion matrix
        self.B = B  # control transition matrix
        self.u = u
        self.Q = Q  # Process uncertainty/noise
        self.H = H  # measurement function
        self.R = R  # measurement uncertainty/noise
        self.X_current = X  # filter state estimate
        self.P_current = P  # covariance matrix
        self.K_gain = None  # Kalman gain

    def run(self, z, verbos=0):
        # z = vector of measurements
        for m in z:
            self.X_current, self.P_current = predict(self.F, self.X_current, self.B, self.u, self.P_current, self.Q)
            self.X_current, self.P_current, self.K_gain = measure(self.P_current, self.H, self.R, self.X_current, m)
            if not verbos==0:
                print('\nAfter measurement:\n', m)
                print('_______________________')
                print('\nX:\n', self.X_current)
                print('_______________________')
                print('\nP:\n', self.P_current)
                if verbos == 2:
                    print('_______________________')
                    print('\nKalman gain:\n', self.K_gain)
        print('Finish!')
        if verbos==0:
            print('\nState Vector is:\n', self.X_current)
            print('_______________________')
            print('\nP matrix is:\n', self.P_current)
            print('_______________________')
            print('\nKalman gain is:\n', self.K_gain)

# Q1:

In [33]:
P = np.matrix([[2**2, 0        ]
              ,[0,         1.2**2]])

deltaT = 1.0
F = np.matrix([[1, deltaT]
              ,[0, 1     ]])

X = np.matrix([[8],
               [5]])

print('After Prediction:')
Q = 0
B = 0
u = 0
X, P = predict(F, X, B, u, P, Q)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print('_________________')

print('After Update:')
R = np.matrix([ 0.5**2])
H = np.matrix([1./0.3048 , 0])
z = np.matrix([43])
X, P, K_gain = measure(P, H, R, X, z)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print("\nKalman Gain is:\n", K_gain)

After Prediction:

P matrix is:
 [[5.44 1.44]
 [1.44 1.44]]

State Vector is:
 [[13.]
 [ 5.]]
_________________
After Update:

P matrix is:
 [[0.02312702 0.00612186]
 [0.00612186 1.06044402]]

State Vector is:
 [[13.10594766]
 [ 5.02804497]]

Kalman Gain is:
 [[0.30350421]
 [0.08033935]]


# Q2:

In [34]:
P = np.matrix([[2**2, 0     ]
              ,[0   , 1.2**2]])

delta_t = 1.0
F = np.matrix([[1, deltaT]
              ,[0, 1     ]])

X = np.matrix([[8],
               [5]])

print('After Prediction:')
Q = 0
B = 0
u = 0
X, P = predict(F, X, B, u, P, Q)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print('_________________')

print('After Update:')
R = np.matrix([[0.5**2, 0   ],
               [0     , 4**2]])
H = np.matrix([[1./0.3048 , 0],
               [0         , 1.0]])
z = np.matrix([[43]
              ,[4]])
X, P, K_gain = measure(P, H, R, X, z)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print("\nKalman Gain is:\n", K_gain)

After Prediction:

P matrix is:
 [[5.44 1.44]
 [1.44 1.44]]

State Vector is:
 [[13.]
 [ 5.]]
_________________
After Update:

P matrix is:
 [[0.02312482 0.00574134]
 [0.00574134 0.99452888]]

State Vector is:
 [[13.10557877]
 [ 4.96414369]]

Kalman Gain is:
 [[0.30347538 0.00035883]
 [0.07534561 0.06215806]]


# Q4:

In [35]:
delta_t = 1.0

P = np.matrix([ [7**2, 0   , 0     , 0],
                [0   , 7**2, 0     , 0],
                [0   , 0   , 100**2, 0],
                [0   , 0   , 0     , 100**2]])

F = np.matrix([ [1, 0, delta_t, 0      ],
                [0, 1, 0      , delta_t],
                [0, 0, 1      , 0      ],
                [0, 0, 0      , 1      ]])

X = np.matrix([ [200],
                [150],
                [0],
                [0]])

H = np.matrix([[1, 0, 0, 0],
               [0, 1, 0, 0]])

R = np.matrix([ [6**2,0   ],
                [0   ,6**2]])

u = np.matrix([ [0],
                [0],
                [5],
                [15]])

B = np.matrix([[0,0,0.5,0  ],
               [0,0,0  ,0.5],
               [0,0,1  ,0  ],
               [0,0,0  ,1  ]])

Q = np.zeros_like(P)

# sensor measurements
z = [np.matrix([[240], [204]]),
     np.matrix([[284], [267]]),
     np.matrix([[334], [344]]),
     np.matrix([[390], [437]]),
     np.matrix([[450], [544]]),
     np.matrix([[516], [667]])]

kalman_filter = KalmanFilter(F, B, u, Q, H, R, X, P)
kalman_filter.run(z, verbos=0)
# X_roi = [515.30128021 667.82815584  67.6560699  130.62964398]

Finish!

State Vector is:
 [[515.30128021]
 [667.82815584]
 [ 67.6560699 ]
 [130.62964398]]
_______________________

P matrix is:
 [[17.059979    0.          4.06497694  0.        ]
 [ 0.         17.059979    0.          4.06497694]
 [ 4.06497694  0.          1.41055982  0.        ]
 [ 0.          4.06497694  0.          1.41055982]]
_______________________

Kalman gain is:
 [[0.47388831 0.        ]
 [0.         0.47388831]
 [0.11291603 0.        ]
 [0.         0.11291603]]
